In [1]:
pip install --upgrade bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%%capture
!pip install bertopic accelerate bitsandbytes adjustText

In [3]:
!pip install --no-deps xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 91.7 MB/s eta 0:00:00:00:0100:01


In [4]:
pip install --upgrade sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from huggingface_hub import notebook_login
from torch import bfloat16
import transformers
from huggingface_hub import HfFolder
from sentence_transformers import SentenceTransformer
from sentence_transformers.quantization import quantize_embeddings
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.decomposition import PCA
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from cuml.cluster import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

In [ ]:
notebook_login()

# Embeddings and selection of text

In [10]:
review = pd.read_csv('/kaggle/input/llm-results/final_llm_analysis_data.csv')

In [ ]:
reviews=review[review['json_Transparency']=='Yes']['json_Transparency_think']

In [128]:
embedding_model = SentenceTransformer('Lajavaness/bilingual-embedding-large-8k', trust_remote_code=True)

In [336]:
embeddings = embedding_model.encode(reviews, show_progress_bar=True)
embeddings_quant=  quantize_embeddings(embeddings,precision='uint8')

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [14]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'
#model_id = 'meta-llama/Llama-3.2-1B'
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=

In [ ]:
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.01,
    repetition_penalty=1.1,
    device_map="auto"
)

In [87]:
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""
example_prompt = """
I have a topic that contains the following texts:

- Flow options to automate things
- Chat bot AI and list of visitors, leads conversation
- How we can interact with clients in real-time.

There are some key phases relevant in each document that will be provided as: 'automate', 'lead conversation', 'interact with clients'

Based on the information about the keywords provided above, please  can you condensate these keywords and get the relevant topic. Make sure to only return the label
Here is the example of a Label: 'Automation marketing leads'
"""
main_prompt = """
[INST] 
    I have a topic that contains the following documents: [DOCUMENTS] 
    The topic is described by the following keywords: '[KEYWORDS]'.
Instructions:
+ Based on the topic information above, please create the label
+ Review your answer and only send the label 
+ Avoid send as your answer texts as 'Sure! Based on the information provided'
+ Only send the label

[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt

# Definition of dimentionality Reduction

In [17]:
no_dim = BaseDimensionalityReduction()

In [338]:
pca_model = PCA(n_components=0.85)

In [19]:
umap_model = UMAP(n_components=6,n_neighbors=15, metric='cosine')

# Clustering defintion

In [20]:
hdbscan_model = HDBSCAN(min_cluster_size=8, prediction_data=True)

In [344]:
kmean_model = KMeans(n_clusters=12)#n_clusters=12

In [22]:
cuml_model = HDBSCAN(min_samples=8, gen_min_span_tree=True, prediction_data=True)

# Vector factorization

In [339]:
vectorizer_model = CountVectorizer(ngram_range=(1,4))

# Representation

In [340]:
# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.30)

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr,
}

# Bertopic execution

In [345]:
topic_model_4 = BERTopic(
                        umap_model= pca_model,#umap_model,no_dim,pca_model
                        embedding_model=embedding_model,#language='french',
                        calculate_probabilities= True,
                        verbose=True,
                        hdbscan_model=kmean_model,#kmean_model,hdbscan_model,cuml_model
                        representation_model=representation_model,#representation_model,
                        vectorizer_model = vectorizer_model,
                        #top_n_words=10,
                        min_topic_size= 10
                       )
topics_process, probs_process = topic_model_4.fit_transform(reviews, embeddings_quant)

2024-11-28 23:24:31,111 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-28 23:24:31,392 - BERTopic - Dimensionality - Completed ✓
2024-11-28 23:24:31,395 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-28 23:24:31,694 - BERTopic - Cluster - Completed ✓
2024-11-28 23:24:31,699 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 12/12 [02:00<00:00, 10.03s/it]
2024-11-28 23:26:36,833 - BERTopic - Representation - Completed ✓


In [346]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model_4.get_topics(full=True)["Llama2"].values()]
topic_model_4.set_topic_labels(llama2_labels)
topics_process_labels=topic_model_4.get_topic_info()

In [347]:
topics_process_labels[['CustomName','Representation','Count','Representative_Docs']]

,CustomName,Representation,Count,Representative_Docs
0,Label: Automation Customer Support,"[to, and, it, of, not, is, are, we, be, in]",116,[i can see there is no option to view data fom...
1,Label: Chatbot Development and Automation,"[to, chat, and, of, it, be, for, that, is, cha...",75,[while user interface is friendly i've had exp...
2,Label: Automation Marketing Leads,"[dislike, nothing, to, there, is, about, to di...",64,"[there is nothing that i dislike about, nothin..."
3,Label: Customer Support Automation,"[to, and, it, is, but, there, are, was, that, ...",63,[moves so quickly with improvements to core pr...
4,Label: Advanced Chatbot Development,"[it, to, is, be, of, and, learning, in, but, t...",60,[- there is significant learning curve to onbo...
5,Label: Bot Development Issues,"[bot, to, is, of, and, in, not, it, can, bots]",57,[somewhat buggy platform and difficult to debu...
6,Label: Marketing Automation Features,"[more, to, would, and, be, of, would be, need,...",55,[additional dashboard customizations and analy...
7,Label: Marketing Automation Pending Implementa...,"[nothing, not, to, yet, my, in, is, it, and, t...",55,[as such nothing and most of functionlity work...
8,Label: Automation Marketing Pricing,"[for, is, to, pricing, it, they, of, be, and, ...",49,[pricing tiers are bit out of reach we would l...
9,Label: Automation Marketing Leads,"[and, to, it, is, sometimes, in, of, that, thi...",47,[ai is always difficult to manage especially s...


In [326]:
topics_process_labels[['CustomName','Representation','Count','Representative_Docs']].to_csv('labels_json_Process.csv')

In [327]:
fig1 = topic_model_4.visualize_heatmap(n_clusters=2,width=1000, height=1000,custom_labels=True)
fig1.write_html("heatmap_json_Process.html")

In [328]:
reduced_embeddings = UMAP(n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings_quant)

In [329]:
fig2 = topic_model_4.visualize_documents(reviews, reduced_embeddings=reduced_embeddings, hide_annotations=False, hide_document_hover=False, custom_labels=True)
fig2.write_html("map_topics_json_Process.html")

In [330]:
hierarchical_topics_process = topic_model_4.hierarchical_topics(reviews)
fig3 = topic_model_4.visualize_hierarchy(hierarchical_topics=hierarchical_topics_process,custom_labels=True)
fig3.write_html("hierachy_topics_json_Process.html")

100%|██████████| 9/9 [00:00<00:00, 345.88it/s]


In [144]:
topic_distr, topic_token_distr = topic_model_4.approximate_distribution(reviews, calculate_tokens=True)

100%|██████████| 1/1 [00:00<00:00, 135.26it/s]


In [145]:
df = topic_model_4.visualize_approximate_distribution(reviews[2], topic_token_distr[2])
df

,Possible,biases,in,AI,powered,chatbot,responses,due,to,lack,of,diverse,training,data
1_chatbot_to_due_due to,0.247,0.430,0.611,0.808,0.808,0.978,1.115,1.248,1.320,1.281,1.288,0.958,0.640,0.326
2_training data_data_training_to,0.000,0.000,0.000,0.000,0.000,0.119,0.303,0.499,0.673,0.730,0.767,0.571,0.397,0.221


In [50]:
df.to_html("example_process_think_word_importance_row_1.html")

In [331]:
fig4=topic_model_4.visualize_barchart()
fig4.write_html("important_words_json_Process.html")

# Final dataset

In [332]:
table_results = {
    'json_Process': reviews,
    'Topic': topics_process
}
final_table = pd.DataFrame(table_results)
merged_table_process = pd.merge(final_table, topics_process_labels[['Topic','CustomName']], on='Topic', how='left')

In [333]:
merged_table_process = merged_table_process.rename(columns={'CustomName': 'Process_topics'})


In [ ]:
merged_table_process.index = reviews_1.index

In [270]:
merged_table_process

,json_Explainability_think,Topic,Explainability_think_topics
4,The chatbot's actions and decisions are transp...,2,Label: Chatbot Decision Making Processes
6,The chatbot provides clear explanations for i...,3,Label: Chatbot Explanations
14,User wants clear explanations of AI's decision...,4,Label: AI Decision Making Transparency
27,Clear explanations provided for chatbot AI dec...,3,Label: Chatbot Explanations
28,Chatbot's transparent responses help users und...,2,Label: Chatbot Decision Making Processes
...,...,...,...
715,User appreciates the ability of the tool to pr...,0,Label: Automation Decision Explanations
721,The tool provides clear explanations for its ...,0,Label: Automation Decision Explanations
723,User finds the bot's responses to be clear and...,1,Label: Explainable Sales Tool
730,The tool provides clear and transparent expla...,0,Label: Automation Decision Explanations


In [334]:
merged_table_process.to_csv('json_Process.csv')